In [3]:
pip install streamlit-jupyter

  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.8.1
    Uninstalling ipywidgets-7.8.1:
      Successfully uninstalled ipywidgets-7.8.1
Note: you may need to restart the kernel to use updated packages.


In [23]:
import requests
import json
import pandas as pd

# List to store property data for DataFrame
property_records = []

def search_properties(api_key, location, property_type="house", limit=3):
    """
    Search for properties using the Realtor API
    
    Parameters:
        api_key (str): Your RapidAPI key
        location (str): City and state, e.g., "New York, NY"
        property_type (str): Type of property (house, condo, apartment)
        limit (int): Maximum number of results to return
    
    Returns:
        dict: JSON response from the API
    """
    url = 'https://realtor16.p.rapidapi.com/search/forsale'
    
    # Parse city and state from location
    #city, state = [x.strip() for x in location.split(',')]
    
    #querystring = {
    #    "city": city,
    #    "state_code": state,
    #    "limit": str(limit),
    #    #"offset": "0",
    #    #"sort": "relevance",
    #    #"property_type": property_type
    #}

    querystring =  {"location":location, 
                    "search_radius":"0",
                    "limit":limit,
                    "property_type":property_type}
    
    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "realtor16.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

def display_and_store_properties(properties_data):
    """Display property information in a readable format based on the actual JSON structure"""
    if not properties_data or 'properties' not in properties_data:
        print("No properties found or invalid data")
        return
    
    for prop in properties_data['properties']:
        print(f"\n{'='*50}")
        
        # Address information
        address = prop.get('location', {}).get('address', {})
        address_line = address.get('line', 'N/A')
        city = address.get('city', 'N/A')
        state = address.get('state_code', 'N/A')
        zip_code = address.get('postal_code', 'N/A')
        
        print(f"Address: {address_line}")
        print(f"City: {city}")
        print(f"State: {state}")
        print(f"Zip: {zip_code}")
        
        # Property details
        description = prop.get('description', {})
        price = prop.get('list_price', 'N/A')
        beds = description.get('beds', 'N/A')
        baths = description.get('baths_consolidated', 'N/A')
        sqft = description.get('sqft', 'N/A')
        lot_sqft = description.get('lot_sqft', 'N/A')
        property_type_desc = description.get('type', 'N/A')
        
        if price != 'N/A':
            print(f"Price: ${price:,}")
        else:
            print(f"Price: {price}")
        print(f"Beds: {beds}")
        print(f"Baths: {baths}")
        print(f"Sq Ft: {sqft}")
        print(f"Lot Size (sq ft): {lot_sqft}")
        print(f"Property Type: {property_type_desc}")
        
        
        # Status flags
        flags = prop.get('flags', {})
        status_flags = []
        if flags.get('is_new_listing'):
            status_flags.append("NEW LISTING")
        if flags.get('is_price_reduced'):
            status_flags.append("PRICE REDUCED")
        if flags.get('is_pending'):
            status_flags.append("PENDING")
        if flags.get('is_foreclosure'):
            status_flags.append("FORECLOSURE")
        if flags.get('is_coming_soon'):
            status_flags.append("COMING SOON")
        if flags.get('is_new_construction'):
            status_flags.append("NEW CONSTRUCTION")
        if flags.get('is_contingent'):
            status_flags.append("CONTINGENT")
            
        status = ', '.join(status_flags) if status_flags else "ACTIVE"
        print(f"Status: {status}")
        
        # Listing information
        listing_id = prop.get('listing_id', 'N/A')
        property_id = prop.get('property_id', 'N/A')
        list_date = prop.get('list_date', 'N/A')
        
        print(f"Listing ID: {listing_id}")
        print(f"Property ID: {property_id}")
        print(f"List Date: {list_date}")
        
        # Images
        primary_image = prop.get('primary_photo', {}).get('href', 'N/A')
        if primary_image != 'N/A':
            print(f"Primary Image: {primary_image}")
        
        # Additional photos count
        photos = prop.get('photos', [])
        additional_photos = len(photos) - 1 if photos and len(photos) > 1 else 0
        if additional_photos > 0:
            print(f"Additional Photos: {additional_photos}")
        
        # Virtual tours
        virtual_tours = prop.get('virtual_tours', [])
        virtual_tour = virtual_tours[0].get('href', 'N/A') if virtual_tours and len(virtual_tours) > 0 else 'N/A'
        if virtual_tour != 'N/A':
            print(f"Virtual Tour: {virtual_tour}")
        
        # Listing agency
        branding = prop.get('branding', [])
        listed_by = branding[0].get('name', 'N/A') if branding and len(branding) > 0 else 'N/A'
        if listed_by != 'N/A':
            print(f"Listed By: {listed_by}")
        
        # Permalink
        permalink = prop.get('permalink', 'N/A')
        listing_url = f"https://www.realtor.com/realestateandhomes-detail/{permalink}" if permalink != 'N/A' else 'N/A'
        print(f"Listing URL: {listing_url}")
        
        # Add data to records list
        property_records.append({
            "Address": address_line,
            "City": city,
            "State": state,
            "Zip": zip_code,
            "Price": price,
            "Beds": beds,
            "Baths": baths,
            "Sq Ft": sqft,
            "Lot Size (sq ft)": lot_sqft,
            "Property Type": property_type_desc,
            "Status": status,
            "Listing ID": listing_id,
            "Property ID": property_id,
            "List Date": list_date,
            "Primary Image": primary_image,
            "Additional Photos": additional_photos,
            "Virtual Tour": virtual_tour,
            "Listed By": listed_by,
            "Listing URL": listing_url
        })
    
    # Create DataFrame from records
    df = pd.DataFrame(property_records)
    
    print(f"\n{'='*50}")
    print(f"Found {len(property_records)} properties.")
    print("Data stored in DataFrame")
    
    # Display DataFrame overview
    print("\nDataFrame Preview:")
    print(f"Shape: {df.shape}")
    print(df.head())
    
    return df


if __name__ == "__main__":
    # Replace with your API key
    API_KEY = "f8652a60a8mshf167b4d824fc77bp1b9522jsnb4348a0da4d6"

    # Make sure pandas is installed
    try:
        import pandas as pd
    except ImportError:
        print("Pandas is not installed. Installing now...")
        import subprocess
        subprocess.check_call(["pip", "install", "pandas"])
        import pandas as pd
    
    # Example search
    location = input("Enter city and state (e.g., Miami, FL): ")
    property_type = input("Enter property type (house, condo, apartment): ") or "house"
    limit = input("Enter maximum number of results (default 10): ") or 10
    
    print(f"\nSearching for {property_type}s in {location}...")
    results = search_properties(API_KEY, location, property_type, int(limit))
    
    if results:
        # Store results in DataFrame and display
        df = display_and_store_properties(results)

Enter city and state (e.g., Miami, FL):  63114
Enter property type (house, condo, apartment):  
Enter maximum number of results (default 10):  1000



Searching for houses in 63114...

Address: 10646 Wurdack Ave
City: Saint Louis
State: MO
Zip: 63114
Price: $249,900
Beds: 3
Baths: 2.5
Sq Ft: 2352
Lot Size (sq ft): 12981
Property Type: single_family
Status: NEW LISTING
Listing ID: 2980980691
Property ID: 8740300994
List Date: 2025-04-25T05:36:41.000000Z
Primary Image: https://ap.rdcpix.com/6dafea3d8fd73d03b9ee6c57aed45803l-m3088196906s.jpg
Additional Photos: 1
Listed By: Garcia Properties
Listing URL: https://www.realtor.com/realestateandhomes-detail/10646-Wurdack-Ave_Saint-Louis_MO_63114_M87403-00994

Address: 3342 Havenbrook Dr
City: Saint Louis
State: MO
Zip: 63114
Price: $175,000
Beds: 4
Baths: 1
Sq Ft: 1387
Lot Size (sq ft): 7344
Property Type: single_family
Status: NEW LISTING
Listing ID: 2980988386
Property ID: 8500617304
List Date: 2025-04-25T13:36:47.000000Z
Primary Image: https://ap.rdcpix.com/84a3f94bfb66a544350c834ac44d2f48l-m4257119223s.jpg
Additional Photos: 1
Listed By: Berkshire Hathaway HomeServices Select Properties

AttributeError: 'NoneType' object has no attribute 'get'

In [22]:
df

,Address,City,State,Zip,Price,Beds,Baths,Sq Ft,Lot Size (sq ft),Property Type,Status,Listing ID,Property ID,List Date,Primary Image,Additional Photos,Virtual Tour,Listed By,Listing URL
0,10646 Wurdack Ave,Saint Louis,MO,63114,249900,3,2.5,2352,12981,single_family,NEW LISTING,2980980691,8740300994,2025-04-25T05:36:41.000000Z,https://ap.rdcpix.com/6dafea3d8fd73d03b9ee6c57...,1,N/A,Garcia Properties,https://www.realtor.com/realestateandhomes-det...
1,3342 Havenbrook Dr,Saint Louis,MO,63114,175000,4,1,1387,7344,single_family,NEW LISTING,2980988386,8500617304,2025-04-25T13:36:47.000000Z,https://ap.rdcpix.com/84a3f94bfb66a544350c834a...,1,N/A,Berkshire Hathaway HomeServices Select Properties,https://www.realtor.com/realestateandhomes-det...
2,2528 Lehigh Ln,Saint Louis,MO,63114,130000,2,1,926,8111,single_family,"NEW LISTING, COMING SOON",2980758789,7608787142,2025-04-19T01:47:41.000000Z,https://ap.rdcpix.com/d59d2a37a883cc1ebbdc8dec...,0,N/A,"Exp Realty, Llc",https://www.realtor.com/realestateandhomes-det...
3,3553 McKibbon Rd,Saint Louis,MO,63114,179900,4,2,1474,7405,single_family,ACTIVE,2980400554,8546465602,2025-04-10T14:14:05.000000Z,https://ap.rdcpix.com/a37579f558f37cdb53bcb8c1...,1,N/A,Your Home Sold Guaranteed Realty,https://www.realtor.com/realestateandhomes-det...
4,8865 Bobb Ave,Saint Louis,MO,63114,125000,2,1,954,13591,single_family,"NEW LISTING, COMING SOON",2980932535,7217118234,2025-04-24T13:36:58.000000Z,https://ap.rdcpix.com/55b75f90c5e6b2696cf00af2...,1,N/A,"EXP Realty, LLC",https://www.realtor.com/realestateandhomes-det...
5,3644 Eminence Blvd,Saint Louis,MO,63114,225000,3,1,1540,5001,single_family,NEW LISTING,2980684873,7082183331,2025-04-17T16:17:20.000000Z,https://ap.rdcpix.com/cd2c7dd0ec0223ea310a103a...,1,https://hommati.tours/tour/2366481?play=no&mls...,Coldwell Banker Realty - Gundaker,https://www.realtor.com/realestateandhomes-det...
6,3602 Roy Ave,Saint John,MO,63114,120000,2,2,1100,5314,single_family,PRICE REDUCED,2975720246,7021663440,2024-12-03T04:43:41.000000Z,https://ap.rdcpix.com/6bd2498168839da44c35d13b...,1,N/A,Sweet Home Realty,https://www.realtor.com/realestateandhomes-det...
7,9605 W Milton Ave,Saint Louis,MO,63114,350000,3,2,3002,17999,single_family,"NEW LISTING, FORECLOSURE",2980841206,8043148818,2025-04-22T14:36:53.000000Z,https://ap.rdcpix.com/79c634b0a114c852bd1f8fcb...,1,N/A,REALTY EXCHANGE,https://www.realtor.com/realestateandhomes-det...
8,1927 Brassie Dr,Saint Louis,MO,63114,185000,3,2,1147,11800,single_family,PENDING,2980655838,8460760416,2025-04-16T21:47:16.000000Z,https://ap.rdcpix.com/e84d31804a92ec6f686828bd...,1,https://mls.kuu.la/share/collection/71Nxk?fs=1...,Jenah Realty Group,https://www.realtor.com/realestateandhomes-det...
9,9124 Arline Ave,Saint Louis,MO,63114,133000,2,1,884,10001,single_family,NEW LISTING,2980800339,7903806385,2025-04-21T15:46:56.000000Z,https://ap.rdcpix.com/c1871600444046327ebb1b74...,1,N/A,Coldwell Banker Realty - Gundaker,https://www.realtor.com/realestateandhomes-det...


In [ ]:
### Let's join the data, calculate the market cap, select the important information, and then sort by the market cap




In [6]:
import requests
import json
import pandas as pd

def search_rental_properties(api_key, location, min_price=None, max_price=None, beds=None, baths=None, limit=10):
    """
    Search for rental properties using the Realtor API
    
    Parameters:
        api_key (str): Your RapidAPI key
        location (str): City and state, e.g., "Kirkwood, MO"
        min_price (int, optional): Minimum rental price
        max_price (int, optional): Maximum rental price
        beds (int, optional): Number of bedrooms
        baths (int, optional): Number of bathrooms
        limit (int, optional): Maximum number of results to return
    
    Returns:
        dict: JSON response from the API
    """
    url = "https://realtor16.p.rapidapi.com/search/forrent"
    
    # Parse city and state from location
    #city, state = [x.strip() for x in location.split(',')]
    
    #querystring = {
    #    "city": city,
    #    "state_code": state,
    #    "limit": str(limit),
    #    "offset": "0",
    #    "sort": "relevance"
    #}

    querystring =  {"location":location, 
                    "search_radius":"0",
                    "limit":limit
                    #"property_type":property_type
                   }

    # Add optional filters if provided
    if min_price:
        querystring["price_min"] = str(min_price)
    if max_price:
        querystring["price_max"] = str(max_price)
    if beds:
        querystring["beds_min"] = str(beds)
    if baths:
        querystring["baths_min"] = str(baths)
    
    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "realtor16.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

def display_and_store_rentals(properties_data):
    """Display rental property information and store in a pandas DataFrame"""
    if not properties_data or 'properties' not in properties_data:
        print("No rental properties found or invalid data")
        return None
    
    # List to store property data for DataFrame
    property_records = []
    
    # Process each property
    for prop in properties_data['properties']:
        try:
            print(f"\n{'='*50}")
            
            # Address information
            address = prop.get('location', {}).get('address', {})
            address_line = address.get('line', 'N/A')
            city = address.get('city', 'N/A')
            state = address.get('state_code', 'N/A')
            zip_code = address.get('postal_code', 'N/A')
            
            print(f"Address: {address_line}")
            print(f"City: {city}")
            print(f"State: {state}")
            print(f"Zip: {zip_code}")
            
            # Property details
            description = prop.get('description', {})
            rent = prop.get('list_price')
            beds = description.get('beds', 'N/A')
            baths = description.get('baths_consolidated', 'N/A')
            sqft = description.get('sqft', 'N/A')
            property_type_desc = description.get('type', 'N/A')
            sub_type = description.get('sub_type', 'N/A')
            
            # Error handling for rent formatting
            if rent is not None:
                try:
                    print(f"Monthly Rent: ${rent:,}")
                    formatted_rent = f"${rent:,}"
                    rent = rent
                except (TypeError, ValueError):
                    print(f"Monthly Rent: ${rent}")
                    formatted_rent = f"${rent}"
                    rent = rent
            else:
                print("Monthly Rent: N/A")
                formatted_rent = "N/A"
                rent = "N/A"
                
            print(f"Beds: {beds}")
            print(f"Baths: {baths}")
            print(f"Sq Ft: {sqft}")
            print(f"Property Type: {property_type_desc}")
            print(f"Sub Type: {sub_type}")
            
            # Status flags
            flags = prop.get('flags', {})
            status_flags = []
            if flags and isinstance(flags, dict):
                if flags.get('is_new_listing'):
                    status_flags.append("NEW LISTING")
                if flags.get('is_pending'):
                    status_flags.append("PENDING")
                
            status = ', '.join(status_flags) if status_flags else "ACTIVE"
            print(f"Status: {status}")
            
            # Price reduction info
            price_reduced = prop.get('price_reduced_amount')
            if price_reduced is not None:
                try:
                    print(f"Price Reduced: ${price_reduced:,}")
                except (TypeError, ValueError):
                    print(f"Price Reduced: ${price_reduced}")
            
            # Listing information
            listing_id = prop.get('listing_id', 'N/A')
            property_id = prop.get('property_id', 'N/A')
            list_date = prop.get('list_date', 'N/A')
            
            print(f"Listing ID: {listing_id}")
            print(f"Property ID: {property_id}")
            if list_date:
                print(f"List Date: {list_date}")
            
            # Pet policy with error handling
            pet_policy = prop.get('pet_policy', {})
            pets_allowed = []
            
            try:
                if pet_policy and isinstance(pet_policy, dict):
                    if pet_policy.get('cats'):
                        pets_allowed.append("Cats")
                    if pet_policy.get('dogs_small'):
                        pets_allowed.append("Small Dogs")
                    if pet_policy.get('dogs_large'):
                        pets_allowed.append("Large Dogs")
                
                pets_string = ', '.join(pets_allowed) if pets_allowed else "No information"
                print(f"Pets Allowed: {pets_string}")
            except Exception as e:
                pets_string = "Error retrieving pet information"
                print(f"Pets Allowed: Error ({str(e)})")
            
            # Get security deposit info from details if available
            security_deposit = "N/A"
            availability_date = "N/A"
            
            try:
                details = prop.get('details', [])
                if details and isinstance(details, list):
                    for detail in details:
                        if not isinstance(detail, dict):
                            continue
                            
                        category = detail.get('category')
                        texts = detail.get('text', [])
                        
                        if not isinstance(texts, list):
                            continue
                            
                        if category == "Rental Info":
                            for text in texts:
                                if isinstance(text, str) and "Security Deposit:" in text:
                                    security_deposit = text.split("Security Deposit:")[1].strip()
                        
                        if category == "Other Property Info":
                            for text in texts:
                                if isinstance(text, str) and "Availability Date:" in text:
                                    availability_date = text.split("Availability Date:")[1].strip()
            except Exception as e:
                print(f"Error processing property details: {str(e)}")
                
            if security_deposit != "N/A":
                print(f"Security Deposit: ${security_deposit}")
            if availability_date != "N/A":
                print(f"Available From: {availability_date}")
            
            # Images
            primary_image = "N/A"
            try:
                primary_photo = prop.get('primary_photo', {})
                if primary_photo and isinstance(primary_photo, dict):
                    primary_image = primary_photo.get('href', 'N/A')
                
                if primary_image != 'N/A':
                    print(f"Primary Image: {primary_image}")
            except Exception as e:
                print(f"Error processing image: {str(e)}")
            
            # Additional photos count
            additional_photos = 0
            try:
                photos = prop.get('photos', [])
                if photos and isinstance(photos, list):
                    additional_photos = max(0, len(photos) - 1)
                
                if additional_photos > 0:
                    print(f"Additional Photos: {additional_photos}")
            except Exception as e:
                print(f"Error counting photos: {str(e)}")
                additional_photos = "Error"
            
            # Virtual tours
            virtual_tour = "N/A"
            try:
                virtual_tours = prop.get('virtual_tours', [])
                if virtual_tours and isinstance(virtual_tours, list) and len(virtual_tours) > 0:
                    if isinstance(virtual_tours[0], dict):
                        virtual_tour = virtual_tours[0].get('href', 'N/A')
                
                if virtual_tour != 'N/A':
                    print(f"Virtual Tour: {virtual_tour}")
            except Exception as e:
                print(f"Error processing virtual tour: {str(e)}")
            
            # Contact info
            contact_phone = "N/A"
            try:
                advertisers = prop.get('advertisers', [])
                if advertisers and isinstance(advertisers, list):
                    for advertiser in advertisers:
                        if not isinstance(advertiser, dict):
                            continue
                            
                        if advertiser.get('type') == "management" and advertiser.get('office'):
                            office = advertiser.get('office')
                            if isinstance(office, dict):
                                phones = office.get('phones', [])
                                if phones and isinstance(phones, list) and len(phones) > 0:
                                    if isinstance(phones[0], dict):
                                        contact_phone = phones[0].get('number', 'N/A')
                
                if contact_phone != 'N/A':
                    print(f"Contact Phone: {contact_phone}")
            except Exception as e:
                print(f"Error processing contact info: {str(e)}")
            
            # Permalink
            permalink = prop.get('permalink', 'N/A')
            listing_url = "N/A"
            try:
                if permalink != 'N/A':
                    listing_url = f"https://www.realtor.com/apartments/{permalink}"
                    print(f"Listing URL: {listing_url}")
            except Exception as e:
                print(f"Error creating listing URL: {str(e)}")
            
            # Add data to records list
            try:
                property_records.append({
                    "Address": address_line,
                    "City": city,
                    "State": state,
                    "Zip": zip_code,
                    "Monthly Rent": formatted_rent,
                    "Monthly Rent v2": rent,
                    "Beds": beds,
                    "Baths": baths,
                    "Sq Ft": sqft,
                    "Property Type": property_type_desc,
                    "Sub Type": sub_type,
                    "Status": status,
                    "Security Deposit": security_deposit,
                    "Available From": availability_date,
                    "Pets Allowed": pets_string,
                    "Listing ID": listing_id,
                    "Property ID": property_id,
                    "List Date": list_date,
                    "Contact Phone": contact_phone,
                    "Primary Image": primary_image,
                    "Additional Photos": additional_photos,
                    "Virtual Tour": virtual_tour,
                    "Listing URL": listing_url
                })
            except Exception as e:
                print(f"Error adding property to records: {str(e)}")
        
        except Exception as e:
            print(f"Error processing property: {str(e)}")
            continue
    
    # Create DataFrame from records
    try:
        df = pd.DataFrame(property_records)
        
        print(f"\n{'='*50}")
        print(f"Found {len(property_records)} rental properties.")
        print("Data stored in DataFrame")
        
        # Display DataFrame overview
        print("\nDataFrame Preview:")
        print(f"Shape: {df.shape}")
        print(df.head())
        
        return df
    except Exception as e:
        print(f"Error creating DataFrame: {str(e)}")
        return None


if __name__ == "__main__":
    # Replace with your API key
    API_KEY = "f8652a60a8mshf167b4d824fc77bp1b9522jsnb4348a0da4d6"
    
    # Make sure pandas is installed
    try:
        import pandas as pd
    except ImportError:
        print("Pandas is not installed. Installing now...")
        import subprocess
        subprocess.check_call(["pip", "install", "pandas"])
        import pandas as pd
    
    # Rental search parameters
    location = input("Enter city and state (e.g., Kirkwood, MO): ")
    
    # Optional parameters
    min_rent = input("Enter minimum rent (or press Enter to skip): ")
    min_rent = int(min_rent) if min_rent.strip() else None
    
    max_rent = input("Enter maximum rent (or press Enter to skip): ")
    max_rent = int(max_rent) if max_rent.strip() else None
    
    beds = input("Enter minimum number of bedrooms (or press Enter to skip): ")
    beds = int(beds) if beds.strip() else None
    
    baths = input("Enter minimum number of bathrooms (or press Enter to skip): ")
    baths = int(baths) if baths.strip() else None
    
    limit = input("Enter maximum number of results (default 10): ") or 10
    
    print(f"\nSearching for rentals in {location}...")
    results = search_rental_properties(
        API_KEY, 
        location, 
        min_price=min_rent, 
        max_price=max_rent, 
        beds=beds, 
        baths=baths, 
        limit=int(limit)
    )
    
    if results:
        # Store results in DataFrame and display
        df_rent = display_and_store_rentals(results)
        
        # Now you can work with the DataFrame
        print("\nYou can now work with the DataFrame 'df'")

Enter city and state (e.g., Kirkwood, MO):  64131
Enter minimum rent (or press Enter to skip):  
Enter maximum rent (or press Enter to skip):  
Enter minimum number of bedrooms (or press Enter to skip):  
Enter minimum number of bathrooms (or press Enter to skip):  
Enter maximum number of results (default 10):  100



Searching for rentals in 64131...

Address: 7208 Wayne City Ave Unit Kansas
City: Kansas City
State: MO
Zip: 64131
Monthly Rent: $1,250
Beds: 2
Baths: 1
Sq Ft: 1195
Property Type: single_family
Sub Type: None
Status: ACTIVE
Listing ID: 2980956828
Property ID: 9499090989
Pets Allowed: Cats, Small Dogs
Security Deposit: $1350
Available From: 2025-04-30
Primary Image: https://ap.rdcpix.com/5bd8939422a2fe6696363ea26dbe1913l-m3201173412s.jpg
Additional Photos: 1
Contact Phone: 8164340652
Listing URL: https://www.realtor.com/apartments/7208-Wayne-City-Ave-Unit-Kansas_Kansas_MO_64131_M94990-90989

Address: 1208 E 85th St
City: Kansas City
State: MO
Zip: 64131
Monthly Rent: $1,495
Beds: 3
Baths: 1
Sq Ft: None
Property Type: single_family
Sub Type: None
Status: ACTIVE
Listing ID: 2980114016
Property ID: 8868447901
Pets Allowed: No information
Primary Image: https://ap.rdcpix.com/a38b9e9871c0aa24909d56fe156b3e85l-m3203262586s.jpg
Contact Phone: 8164060200
Listing URL: https://www.realtor.com/ap

In [7]:
df_rent

,Address,City,State,Zip,Monthly Rent,Monthly Rent v2,Beds,Baths,Sq Ft,Property Type,...,Available From,Pets Allowed,Listing ID,Property ID,List Date,Contact Phone,Primary Image,Additional Photos,Virtual Tour,Listing URL
0,7208 Wayne City Ave Unit Kansas,Kansas City,MO,64131,"$1,250",1250,2.0,1,1195.0,single_family,...,2025-04-30,"Cats, Small Dogs",2980956828,9499090989,None,8164340652,https://ap.rdcpix.com/5bd8939422a2fe6696363ea2...,1,N/A,https://www.realtor.com/apartments/7208-Wayne-...
1,1208 E 85th St,Kansas City,MO,64131,"$1,495",1495,3.0,1,NaN,single_family,...,N/A,No information,2980114016,8868447901,None,8164060200,https://ap.rdcpix.com/a38b9e9871c0aa24909d56fe...,0,N/A,https://www.realtor.com/apartments/1208-E-85th...
2,429 E Meyer Blvd,Kansas City,MO,64131,N/A,N/A,NaN,None,NaN,apartment,...,N/A,No information,2969518252,7547727858,2024-07-13T00:35:56.000000Z,None,https://ar.rdcpix.com/0247e8fa5c48cbeb4d0e5dc1...,0,N/A,https://www.realtor.com/apartments/429-E-Meyer...
3,9800 Cherry St,Kansas City,MO,64131,N/A,N/A,NaN,None,NaN,apartment,...,N/A,No information,2977945453,7431683302,2025-02-04T08:26:01.000000Z,None,https://ar.rdcpix.com/d46d3c65991c6f3fb0a39ca6...,0,N/A,https://www.realtor.com/apartments/9800-Cherry...
4,9508-9510 Harrison St,Kansas City,MO,64131,N/A,N/A,NaN,None,NaN,apartment,...,N/A,No information,2966247230,9847983162,2025-04-15T00:23:30.000000Z,None,https://ar.rdcpix.com/3fd763a7eddb63381493654e...,0,N/A,https://www.realtor.com/apartments/9508-9510-H...
5,655 E Minor Dr,Kansas City,MO,64131,N/A,N/A,NaN,None,NaN,apartment,...,N/A,Cats,2963763379,7406867057,None,None,https://ar.rdcpix.com/b3a0bfa692f687a13beab660...,1,N/A,https://www.realtor.com/apartments/655-E-Minor...
6,8101 Campbell St,Kansas City,MO,64131,N/A,N/A,NaN,None,NaN,apartment,...,N/A,Cats,2873709244,7091202757,None,None,https://ar.rdcpix.com/545cdc4c9819359a09db1f56...,1,N/A,https://www.realtor.com/apartments/8101-Campbe...
7,8321 Holmes Rd,Kansas City,MO,64131,"$1,495",1495,3.0,1.5,1368.0,single_family,...,N/A,"Cats, Small Dogs, Large Dogs",2980801485,8941332044,2025-04-21T16:15:24.000000Z,N/A,https://ap.rdcpix.com/614a0f85278516fc9861ed86...,1,N/A,https://www.realtor.com/apartments/8321-Holmes...
8,1127 E 75th St,Kansas City,MO,64131,"$2,500",2500,3.0,3,1672.0,single_family,...,2025-02-01,Cats,2976538758,8241560217,2024-12-20T00:45:10.000000Z,N/A,https://ap.rdcpix.com/2b023482ca5e13dce1b942b5...,1,N/A,https://www.realtor.com/apartments/1127-E-75th...
9,907 E 78th St,Kansas City,MO,64131,"$1,940",1940,3.0,2,1966.0,single_family,...,2024-10-04,No information,2973292676,7589434745,2024-10-05T00:13:03.000000Z,9138026533,https://ap.rdcpix.com/7b288697109cfcbd30d30f7b...,1,N/A,https://www.realtor.com/apartments/907-E-78th-...


In [9]:
# Assuming your dataframe is called 'df'
# Group by the three columns and calculate the summary statistics for Monthly Rent
df_rent['Monthly Rent v2'] = pd.to_numeric(df_rent['Monthly Rent v2'], errors='coerce')

rent_summary = df_rent.groupby(['Property Type', 'Beds', 'Baths'])['Monthly Rent v2'].agg([
    ('Count', 'count'),
    ('Min Rent', 'min'),
    ('Median Rent', 'median'),
    ('Max Rent', 'max')
]).reset_index()

# Display the results
print(rent_summary)

    Property Type  Beds Baths  Count  Min Rent  Median Rent  Max Rent
0       apartment   1.0   0.5      1     855.0        855.0     855.0
1       apartment   1.0     1      1    1150.0       1150.0    1150.0
2       apartment   2.0     1      1    1345.0       1345.0    1345.0
3          condos   1.0     1      1    1100.0       1100.0    1100.0
4          condos   2.0     1      1     995.0        995.0     995.0
5   single_family   2.0     1      3    1049.0       1250.0    1345.0
6   single_family   3.0     1      2    1495.0       1495.0    1495.0
7   single_family   3.0   1.5      1    1495.0       1495.0    1495.0
8   single_family   3.0     2      1    1940.0       1940.0    1940.0
9   single_family   3.0   2.5      1    1995.0       1995.0    1995.0
10  single_family   3.0     3      1    2500.0       2500.0    2500.0
11  single_family   4.0     2      1    1675.0       1675.0    1675.0
12  single_family   6.0     2      1     350.0        350.0     350.0
13      townhomes   

In [13]:
# Then perform the left join to keep all rows from df
result_df_test = df.merge(
    rent_summary[['Property Type', 'Beds', 'Baths', 'Median Rent']], 
    on=['Property Type', 'Beds', 'Baths'],
    how='left'
)

In [19]:
# 1. Create Estimated Annual Rent column
result_df_test['Estimated Annual Rent'] = result_df_test['Median Rent'] * 12

# 2. Create Projected Expenses column
result_df_test['Projected Expenses'] = result_df_test['Estimated Annual Rent'] * 0.4

# 3. Create NOI (Net Operating Income) column
result_df_test['NOI'] = result_df_test['Estimated Annual Rent'] - result_df_test['Projected Expenses'] 

# 4. Create Cap Rate column (as a percentage)
# Note: Using the Price column from the original df as Sale Price
result_df_test['Cap Rate'] = 100 * result_df_test['NOI'] / result_df_test['Price']

# Sort by Cap Rate in descending order (highest first)
result_df_test = result_df_test.sort_values(by='Cap Rate', ascending=False)

In [20]:
result_df_test

,Address,City,State,Zip,Price,Beds,Baths,Sq Ft,Lot Size (sq ft),Property Type,...,Primary Image,Additional Photos,Virtual Tour,Listed By,Listing URL,Median Rent,Estimated Annual Rent,Projected Expenses,NOI,Cap Rate
39,401 Meacham St,Saint Louis,MO,63122,118750,3.0,1,912.0,4700.0,single_family,...,https://ap.rdcpix.com/bd25d6c420585bff179c88f3...,1,N/A,REALTY EXCHANGE,https://www.realtor.com/realestateandhomes-det...,1495.0,17940.0,7176.0,10764.0,9.064421
41,13339 Rosebank Ln,Saint Louis,MO,63122,125000,3.0,1,1575.0,27878.0,single_family,...,https://ap.rdcpix.com/53f7d99348f76e84bd37a0c1...,1,N/A,The Hermann London Group Llc,https://www.realtor.com/realestateandhomes-det...,1495.0,17940.0,7176.0,10764.0,8.611200
129,1926 Greenpoint Dr Apt 201,Kirkwood,MO,63122,105900,1.0,1,657.0,2265.0,condos,...,https://ap.rdcpix.com/aec8ce5a0c6c9e9b1105084d...,0,N/A,Element Group,https://www.realtor.com/realestateandhomes-det...,1100.0,13200.0,5280.0,7920.0,7.478754
37,1929 Greenpoint Dr Apt 301,Saint Louis,MO,63122,119000,1.0,1,753.0,2274.0,condos,...,https://ap.rdcpix.com/1715a45dcd54ab9266f246f1...,1,N/A,Allen Brake Real Estate,https://www.realtor.com/realestateandhomes-det...,1100.0,13200.0,5280.0,7920.0,6.655462
160,1934 Greenpoint Dr Apt 304,Saint Louis,MO,63122,119900,1.0,1,657.0,2265.0,condos,...,https://ap.rdcpix.com/5b2499786b2817d77a23c393...,1,N/A,Keller Williams Chesterfield,https://www.realtor.com/realestateandhomes-det...,1100.0,13200.0,5280.0,7920.0,6.605505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,132 E Monroe Ave Unit 308,Saint Louis,MO,63122,1191400,2.0,2.5,2072.0,NaN,single_family,...,https://ap.rdcpix.com/0cd47196b6830babbb52615a...,1,N/A,"Mehlman Homes Realty, Llc",https://www.realtor.com/realestateandhomes-det...,NaN,NaN,NaN,NaN,NaN
182,132 E Monroe Ave Unit 302,Saint Louis,MO,63122,1423835,3.0,3.5,2393.0,NaN,single_family,...,https://ap.rdcpix.com/9ecc1dd8c00ed468ea041bcc...,1,N/A,"Mehlman Homes Realty, Llc",https://www.realtor.com/realestateandhomes-det...,NaN,NaN,NaN,NaN,NaN
183,132 E Monroe Ave Unit 103,Saint Louis,MO,63122,1170125,2.0,2.5,2035.0,NaN,single_family,...,https://ap.rdcpix.com/a35e2bc865c8d30ce7d292c1...,1,N/A,"Mehlman Homes Realty, Llc",https://www.realtor.com/realestateandhomes-det...,NaN,NaN,NaN,NaN,NaN
184,649 S Sappington Blvd Lots 2 & 3,Oakland,MO,63122,199900,NaN,None,NaN,1394.0,land,...,https://ap.rdcpix.com/a89d7f51ed3624becf8cb984...,1,N/A,"Robert Biggs, Realtors",https://www.realtor.com/realestateandhomes-det...,NaN,NaN,NaN,NaN,NaN


In [5]:
import streamlit as st
from streamlit_jupyter import StreamlitPatcher
import requests
import pandas as pd
import json

# Set page title and description
st.title('Realtor.com Rental Property Finder')
st.write('Enter a zip code to find available rental properties in that area.')

# Create zip code input field
zip_code = st.text_input('Enter zip code:', placeholder='e.g., 90210')

# Optional: Add validation for zip code format
if zip_code and not (zip_code.isdigit() and len(zip_code) == 5):
    st.warning('Please enter a valid 5-digit zip code')

# Create a search button
search_button = st.button('Search Properties')

# Only process when the search button is clicked and zip code is valid
if search_button and zip_code and zip_code.isdigit() and len(zip_code) == 5:
    with st.spinner('Fetching rental properties...'):
        # Call API function
        results = 'hello world'

        df = df_rent

# Display as interactive table
        st.subheader('Available Rental Properties')
        st.dataframe(df)
                
        # Optional: Add filters
        st.subheader('Filter Results')
        min_beds = st.slider('Minimum Bedrooms', 0, 5, 0)
        max_price = st.slider('Maximum Price ($)', 500, 10000, 10000, step=100)
            

In [ ]:
import streamlit as st
import requests
import pandas as pd
import json

# Set page title and description
st.title('Realtor.com Rental Property Finder')
st.write('Enter a zip code to find available rental properties in that area.')

# Create zip code input field
zip_code = st.text_input('Enter zip code:', placeholder='e.g., 90210')

# Optional: Add validation for zip code format
if zip_code and not (zip_code.isdigit() and len(zip_code) == 5):
    st.warning('Please enter a valid 5-digit zip code')

# Create a search button
search_button = st.button('Search Properties')

# Function to call Realtor.com API
#def get_rental_properties(zip_code):
#    # Replace this with your actual API request code
#    url = f"https://realtor.p.rapidapi.com/properties/v2/list-for-rent"
#    
#    querystring = {
#        "postal_code": zip_code,
#        "limit": "20",
#        "offset": "0",
#        "sort": "relevance"
#    }
#    
#    headers = {
#        "X-RapidAPI-Key": st.secrets["RAPIDAPI_KEY"],  # Store API key in Streamlit secrets
#        "X-RapidAPI-Host": "realtor.p.rapidapi.com"
#    }
#    
#    try:
#        response = requests.get(url, headers=headers, params=querystring)
#        response.raise_for_status()  # Raise exception for 4XX/5XX responses
#        return response.json()
#    except requests.exceptions.RequestException as e:
#        st.error(f"API Error: {e}")
#        return None

# Only process when the search button is clicked and zip code is valid
if search_button and zip_code and zip_code.isdigit() and len(zip_code) == 5:
    with st.spinner('Fetching rental properties...'):
        # Call API function
        #results = get_rental_properties(zip_code)
        results = 'hello world'
        
        if results and 'properties' in results:
            properties = results['properties']
            
            if not properties:
                st.info(f"No rental properties found in zip code {zip_code}")
            else:
                st.success(f"Found {len(properties)} rental properties in {zip_code}")
                
                # Extract relevant information and create DataFrame
                property_data = []
                for prop in properties:
                    property_info = {
                        'Address': f"{prop.get('address', {}).get('line', 'N/A')}, {prop.get('address', {}).get('city', 'N/A')}, {prop.get('address', {}).get('state_code', 'N/A')}",
                        'Price': prop.get('price', 'N/A'),
                        'Beds': prop.get('beds', 'N/A'),
                        'Baths': prop.get('baths', 'N/A'),
                        'Sqft': prop.get('building_size', {}).get('size', 'N/A'),
                        'Property Type': prop.get('prop_type', 'N/A'),
                        'URL': prop.get('rdc_web_url', '#')
                    }
                    property_data.append(property_info)
                
                df = pd.DataFrame(property_data)
                
                # Display as interactive table
                st.subheader('Available Rental Properties')
                st.dataframe(df)
                
                # Optional: Add filters
                st.subheader('Filter Results')
                min_beds = st.slider('Minimum Bedrooms', 0, 5, 0)
                max_price = st.slider('Maximum Price ($)', 500, 10000, 10000, step=100)
                
        results = get_rental_properties(zip_code)
        
        if results and 'properties' in results:
            properties = results['properties']
            
            if not properties:
                st.info(f"No rental properties found in zip code {zip_code}")
            else:
                st.success(f"Found {len(properties)} rental properties in {zip_code}")
                
                # Extract relevant information and create DataFrame
                property_data = []
                for prop in properties:
                    property_info = {
                        'Address': f"{prop.get('address', {}).get('line', 'N/A')}, {prop.get('address', {}).get('city', 'N/A')}, {prop.get('address', {}).get('state_code', 'N/A')}",
                        'Price': prop.get('price', 'N/A'),
                        'Beds': prop.get('beds', 'N/A'),
                        'Baths': prop.get('baths', 'N/A'),
                        'Sqft': prop.get('building_size', {}).get('size', 'N/A'),
                        'Property Type': prop.get('prop_type', 'N/A'),
                        'URL': prop.get('rdc_web_url', '#')
                    }
                    property_data.append(property_info)
                
                df = pd.DataFrame(property_data)
                
                # Display as interactive table
                st.subheader('Available Rental Properties')
                st.dataframe(df)
                
                # Optional: Add filters
                st.subheader('Filter Results')
                min_beds = st.slider('Minimum Bedrooms', 0, 5, 0)
                max_price = st.slider('Maximum Price ($)', 500, 10000, 10000, step=100)
                
        results = get_rental_properties(zip_code)
        
        if results and 'properties' in results:
            properties = results['properties']
            
            if not properties:
                st.info(f"No rental properties found in zip code {zip_code}")
            else:
                st.success(f"Found {len(properties)} rental properties in {zip_code}")
                
                # Extract relevant information and create DataFrame
                property_data = []
                for prop in properties:
                    property_info = {
                        'Address': f"{prop.get('address', {}).get('line', 'N/A')}, {prop.get('address', {}).get('city', 'N/A')}, {prop.get('address', {}).get('state_code', 'N/A')}",
                        'Price': prop.get('price', 'N/A'),
                        'Beds': prop.get('beds', 'N/A'),
                        'Baths': prop.get('baths', 'N/A'),
                        'Sqft': prop.get('building_size', {}).get('size', 'N/A'),
                        'Property Type': prop.get('prop_type', 'N/A'),
                        'URL': prop.get('rdc_web_url', '#')
                    }
                    property_data.append(property_info)
                
                df = pd.DataFrame(property_data)
                
                # Display as interactive table
                st.subheader('Available Rental Properties')
                st.dataframe(df)
                
                # Optional: Add filters
                st.subheader('Filter Results')
                min_beds = st.slider('Minimum Bedrooms', 0, 5, 0)
                max_price = st.slider('Maximum Price ($)', 500, 10000, 10000, step=100)
                
                # Apply filters
                filtered_df = df[
                    (df['Beds'] >= min_beds) & 
                    (df['Price'] <= max_price)
                ]
                
                st.subheader('Filtered Properties')
                st.dataframe(filtered_df)
                
                # Visualization
                st.subheader('Price Distribution')
                if 'Price' in df.columns and df['Price'].notna().any():
                    # Convert prices to numeric if they're not already
                    if not pd.api.types.is_numeric_dtype(df['Price']):
                        df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
                    
                    st.bar_chart(df['Price'].value_counts().sort_index())
        else:
            st.error("Failed to retrieve data from Realtor.com API")

# Add instructions
with st.expander("How to use this app"):
    st.write("""
    1. Enter a valid 5-digit zip code in the input field
    2. Click the 'Search Properties' button
    3. View the retrieved rental properties
    4. Use the filters to narrow down results by bedrooms and price
    """)

# Note about API key
st.sidebar.info("""
**Note:** This app requires a valid Realtor.com API key.
Store your API key in Streamlit secrets.
""")